<a href="https://colab.research.google.com/github/sightes/BIGDATA-UBO2022/blob/main/clase_03_12_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive 
drive.mount("/content/gdrive")
gdrive='content/gdrive/MyDrive/UBO-BIGDATA2'
     

Mounted at /content/gdrive


In [2]:
%%capture
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget 'https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz' -O 'spark-3.3.1-bin-hadoop3.tgz'
!tar -xvf /content/spark-3.3.1-bin-hadoop3.tgz
!pip install -q findspark 
     

In [3]:
import os 
os.environ['JAVA_HOME']='/usr/lib/jvm/java-8-openjdk-amd64'
os.environ['SPARK_HOME']='/content/spark-3.3.1-bin-hadoop3'

In [4]:
import findspark 
findspark.init()
from pyspark.sql import SparkSession
spark=SparkSession.builder.master("local[*]").appName("ejemplo1").getOrCreate()

In [ ]:
df=spark.read.csv('/'+gdrive+'/WorldCupPlayers.csv',inferSchema=True,header=True)
df.show()

In [10]:
df.printSchema()

root
 |-- RoundID: integer (nullable = true)
 |-- MatchID: integer (nullable = true)
 |-- Team Initials: string (nullable = true)
 |-- Coach Name: string (nullable = true)
 |-- Line-up: string (nullable = true)
 |-- Shirt Number: integer (nullable = true)
 |-- Player Name: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Event: string (nullable = true)



In [11]:
df.count()

37784

In [24]:
df.select('Player Name','Coach Name').distinct().count()

9078

In [14]:
df.describe('Shirt Number').show()

+-------+------------------+
|summary|      Shirt Number|
+-------+------------------+
|  count|             37784|
|   mean|10.726021596442939|
| stddev| 6.960138422882888|
|    min|                 0|
|    max|                23|
+-------+------------------+



In [19]:
df.filter(df['Team Initials']=="NED").show()

#equivalente en pandas : df.iloc[np.where(df['Team Initials']=="NED")]

+-------+-------+-------------+--------------------+-------+------------+--------------------+--------+-----+
|RoundID|MatchID|Team Initials|          Coach Name|Line-up|Shirt Number|         Player Name|Position|Event|
+-------+-------+-------------+--------------------+-------+------------+--------------------+--------+-----+
|    204|   1133|          NED|GLENDENNING Bob (...|      S|           0|Gejus VAN DER MEULEN|      GK| null|
|    204|   1133|          NED|GLENDENNING Bob (...|      S|           0|      Henk PELLIKAAN|    null| null|
|    204|   1133|          NED|GLENDENNING Bob (...|      S|           0|           Kick SMIT|    null| G29'|
|    204|   1133|          NED|GLENDENNING Bob (...|      S|           0|       Puck VAN HEEL|       C| null|
|    204|   1133|          NED|GLENDENNING Bob (...|      S|           0|   Willem ANDERIESEN|    null| null|
|    204|   1133|          NED|GLENDENNING Bob (...|      S|           0|         Beb BAKHUYS|    null| null|
|    204| 

In [29]:
df.createOrReplaceTempView("temp_view")
spark.sql("select * from temp_view where MatchID=1133").show()

+-------+-------+-------------+--------------------+-------+------------+--------------------+--------+--------+
|RoundID|MatchID|Team Initials|          Coach Name|Line-up|Shirt Number|         Player Name|Position|   Event|
+-------+-------+-------------+--------------------+-------+------------+--------------------+--------+--------+
|    204|   1133|          SUI| MUELLER Henry (SUI)|      S|           0|     Frank SECHEHAYE|      GK|    null|
|    204|   1133|          NED|GLENDENNING Bob (...|      S|           0|Gejus VAN DER MEULEN|      GK|    null|
|    204|   1133|          SUI| MUELLER Henry (SUI)|      S|           0|    Willy VON KAENEL|    null|    null|
|    204|   1133|          NED|GLENDENNING Bob (...|      S|           0|      Henk PELLIKAAN|    null|    null|
|    204|   1133|          SUI| MUELLER Henry (SUI)|      S|           0|       Walter WEILER|    null|    null|
|    204|   1133|          NED|GLENDENNING Bob (...|      S|           0|           Kick SMIT|  

In [30]:
df.printSchema()

root
 |-- RoundID: integer (nullable = true)
 |-- MatchID: integer (nullable = true)
 |-- Team Initials: string (nullable = true)
 |-- Coach Name: string (nullable = true)
 |-- Line-up: string (nullable = true)
 |-- Shirt Number: integer (nullable = true)
 |-- Player Name: string (nullable = true)
 |-- Position: string (nullable = true)
 |-- Event: string (nullable = true)



### Stock Data 




In [31]:
df=spark.read.csv('/'+gdrive+'/stocks_price_final.csv',sep=',',header=True)
df.show()

+---+------+----------+---------+---------+---------+---------+-------+---------+----------+-------------+--------------------+--------+
|_c0|symbol|      date|     open|     high|      low|    close| volume| adjusted|market.cap|       sector|            industry|exchange|
+---+------+----------+---------+---------+---------+---------+-------+---------+----------+-------------+--------------------+--------+
|  1|   TXG|2019-09-12|       54|       58|       51|    52.75|7326300|    52.75|    $9.31B|Capital Goods|Biotechnology: La...|  NASDAQ|
|  2|   TXG|2019-09-13|    52.75|   54.355|49.150002|    52.27|1025200|    52.27|    $9.31B|Capital Goods|Biotechnology: La...|  NASDAQ|
|  3|   TXG|2019-09-16|52.450001|       56|52.009998|55.200001| 269900|55.200001|    $9.31B|Capital Goods|Biotechnology: La...|  NASDAQ|
|  4|   TXG|2019-09-17|56.209999|60.900002|   55.423|56.779999| 602800|56.779999|    $9.31B|Capital Goods|Biotechnology: La...|  NASDAQ|
|  5|   TXG|2019-09-18|56.849998|    62.2

In [32]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- symbol: string (nullable = true)
 |-- date: string (nullable = true)
 |-- open: string (nullable = true)
 |-- high: string (nullable = true)
 |-- low: string (nullable = true)
 |-- close: string (nullable = true)
 |-- volume: string (nullable = true)
 |-- adjusted: string (nullable = true)
 |-- market.cap: string (nullable = true)
 |-- sector: string (nullable = true)
 |-- industry: string (nullable = true)
 |-- exchange: string (nullable = true)



In [ ]:
from pyspark.sql.types import *
dataschema=[
StructField('c0', IntegerType(),True),
StructField('symbol', StringType(),True),
StructField('date', DateType(),True),
StructField('open', DoubleType(),True),
StructField('high', DoubleType(),True),
StructField('low', DoubleType(),True),
StructField('close', DoubleType(),True),
StructField('volume', IntegerType(),True),
StructField('adjusted', DoubleType(),True),
StructField('market.cap', StringType(),True),
StructField('sector', StringType(),True),
StructField('industry', StringType(),True),
StructField('exchange', StringType(),True)]
final_struct= StructType(fields=dataschema)



In [43]:
df=spark.read.csv('/'+gdrive+'/stocks_price_final.csv',sep=',',header=True,schema=final_struct)
df.printSchema()

root
 |-- c0: integer (nullable = true)
 |-- symbol: string (nullable = true)
 |-- date: date (nullable = true)
 |-- open: double (nullable = true)
 |-- high: double (nullable = true)
 |-- low: double (nullable = true)
 |-- close: double (nullable = true)
 |-- volume: integer (nullable = true)
 |-- adjusted: double (nullable = true)
 |-- market.cap: string (nullable = true)
 |-- sector: string (nullable = true)
 |-- industry: string (nullable = true)
 |-- exchange: string (nullable = true)



In [46]:
df=df.withColumnRenamed('market.cap','market_cap')

In [47]:
df.na.drop()

DataFrame[c0: int, symbol: string, date: date, open: double, high: double, low: double, close: double, volume: int, adjusted: double, market_cap: string, sector: string, industry: string, exchange: string]

In [53]:
df.na.fill(0)

DataFrame[c0: int, symbol: string, date: date, open: double, high: double, low: double, close: double, volume: int, adjusted: double, market_cap: string, sector: string, industry: string, exchange: string]

In [48]:
df.dtypes

[('c0', 'int'),
 ('symbol', 'string'),
 ('date', 'date'),
 ('open', 'double'),
 ('high', 'double'),
 ('low', 'double'),
 ('close', 'double'),
 ('volume', 'int'),
 ('adjusted', 'double'),
 ('market_cap', 'string'),
 ('sector', 'string'),
 ('industry', 'string'),
 ('exchange', 'string')]

In [50]:
df.first()

Row(c0=1, symbol='TXG', date=datetime.date(2019, 9, 12), open=54.0, high=58.0, low=51.0, close=52.75, volume=7326300, adjusted=52.75, market_cap='$9.31B', sector='Capital Goods', industry='Biotechnology: Laboratory Analytical Instruments', exchange='NASDAQ')

In [52]:
df.columns

['c0',
 'symbol',
 'date',
 'open',
 'high',
 'low',
 'close',
 'volume',
 'adjusted',
 'market_cap',
 'sector',
 'industry',
 'exchange']